In [ ]:
import pandas as pd
import warnings


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
bikes=pd.read_csv('day.csv')   #importing data set

#  Reading and Understanding the Data

In [ ]:
bikes.head(5)   #Inspect first few rows

In [ ]:
#check the shape
bikes.shape

In [ ]:
bikes.info()  

In [ ]:
bikes.describe()          #description  of numerical data

In [ ]:
#checking for null values
bikes.isnull().sum()

### Dropping columns not useful for analysis

1. 'dteday' consists of the date on which records are takens and are unique. it doesnt contribute to our analysis\
2. 'instant' is just a row instance identifier.\
3. 'registered' and 'casual' variables are not available at the time of prediction.And we see 'casual'+'registered'=target column, which leads to data leakage.

In [ ]:
bikes.drop(['instant','dteday','registered','casual'],axis=1,inplace=True)

# Visualising the Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#checking the trends of temp, atemp, hum, windspeed with target column
sns.pairplot(bikes[['temp','hum','atemp','windspeed','cnt']])
plt.show();

1. we see high correlation of temp with atemp\
2. we see temp and atemp has somewhat linear relation with the target column 'cnt\


In [ ]:
# checking for outliers
plt.figure(figsize=(20,5))
plt.subplot(1,4,1)
sns.boxplot(bikes['temp'])
plt.subplot(1,4,2)
sns.boxplot(bikes['atemp'])
plt.subplot(1,4,3)
sns.boxplot(bikes['hum'])
plt.subplot(1,4,4)
sns.boxplot(bikes['windspeed'])
plt.show()


In [ ]:
# month wise bike demand
plt.figure(figsize=(15,12))
sns.boxplot(y='cnt',x='mnth',data=bikes,hue='yr')
plt.show()

1. There is considerable rise in the number of bike users every month in 2019 than the last year \
2. the number of bike users increase every month from januray to june and from july to dec there is seen a decrease in users every year.

In [ ]:
# comparing bike demands year wise, on weekdays
sns.boxplot(y='cnt',x='weekday',data=bikes,hue='yr')
plt.grid()
plt.show()

In [ ]:
# comparing bike demands year wise, on weekdays
sns.boxplot(y='cnt',x='holiday',data=bikes,hue='yr')
plt.show()

In [ ]:
# scatter plot showing trends of temperature on count of rented bikes over  the year 2018 and 2019
sns.scatterplot(x='temp',y='cnt',data=bikes,hue='yr')
plt.show()

1. higher temperature indicates more bike users

# Data Preparation

In [ ]:
bikes.head(4)

   **CONVERTING 'season' , 'weathersit' , 'mnth' , 'weekday'  to  categorical  column**

In [ ]:
# converting 'season' , 'weathersit' , 'mnth' , 'weekday'  to categorical columns.

bikes.season.replace({1:"spring", 2:"summer", 3:"fall", 4:"winter"},inplace = True)

bikes.weathersit.replace({1:'good',2:'moderate',3:'bad',4:'severe'},inplace = True)

bikes.mnth.replace({1: 'jan',2: 'feb',3: 'mar',4: 'apr',5: 'may',6: 'jun',
                  7: 'jul',8: 'aug',9: 'sept',10: 'oct',11: 'nov',12: 'dec'},inplace=True)

bikes.weekday.replace({0: 'sun',1: 'mon',2: 'tue',3: 'wed',4: 'thu',5: 'fri',6: 'sat'},inplace=True)
bikes.head()

**Creating dummy variables for 'season' , 'weathersit' , 'mnth' , 'weekday' columns**

In [ ]:
# creating dummy variables
dumies=pd.get_dummies(bikes[['season','mnth','weekday','weathersit']],drop_first=True,prefix_sep='-')   
# dropping the first to avoid multicolinearity

In [ ]:
# concating the dummy variables to the bikes data frame.
bikes=pd.concat([dumies,bikes],axis=1) 
bikes.head(5)

In [ ]:
# dropping the columns 'weekday','season','weathersit','mnth' as dummies of these have been created
bikes.drop(['weekday','season','weathersit','mnth'],axis=1,inplace=True)

In [ ]:
bikes.columns   # taking a look at the columns after concating

# Splitting the Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
bikes1=bikes.copy()

In [ ]:
#splitting into train and test set
train,test=train_test_split(bikes1,test_size=0.30,random_state=47)  


# feature scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

It is important to have all the variables on the same scale for the model to be easily interpretable. We can use standardization or normalization so that the units of the coefficients obtained are all on the same scale.


In [ ]:
scaler=MinMaxScaler()   

In [ ]:
x=['temp','atemp','hum','windspeed']  # list of numerical columns we want to scale

In [ ]:
train[x]=scaler.fit_transform(train[x]) # scalling the train set

In [ ]:
train

In [ ]:
test[x]=scaler.transform(test[x])  #scalling the test data

In [ ]:
test

In [ ]:
# checking the correlation of different columns 
plt.figure(figsize = (30, 30))
sns.heatmap(train.corr(), annot = True, cmap="YlGnBu")
plt.show()

cnt column has linear realtionship with atemp


In [ ]:
#checking the relationship of 'atemp' column with the target variable
sns.regplot(x='atemp',y='cnt',data=train)
plt.show()

### Now we will seperate the target variable from the test and train dataframes

In [ ]:
y_train=train.iloc[:,-1] 
x_train=train.iloc[:,:-1]

In [ ]:
y_test=test.iloc[:,-1]
x_test=test.iloc[:,:-1]

In [ ]:
y_train=pd.DataFrame(y_train)

In [ ]:
y_test=pd.DataFrame(y_test)

# Building model using statsmodel, for the detailed statistics

In [ ]:
import statsmodels.api as sm

### Creating model using all the independent variables

In [ ]:
x_train2=sm.add_constant(x_train)    # adding a constant variable

In [ ]:
lm=sm.OLS(y_train,x_train2).fit()    # running the linear model
print(lm.summary())

#### OBSERVATIONS
1. **The R-squared value is 0.847.**
2. **But we have high p-values for 'weekday-thu','mnth-aug','season-summer','mnth-jun','mnth-mar','mnth-oct','weekday-sun','atemp'columns which shows these columns are not significant.   
Dropping the insinificant columns**

In [ ]:
# list of columns to drop
x=['weekday-thu','mnth-aug','season-summer','mnth-jun','mnth-mar','mnth-oct','weekday-sun','atemp']

### Dropping the variable and updating the model

In [ ]:
x_train3=x_train.drop(x,axis=1)     #dropping the columns with high p-value

In [ ]:
x_train4=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train4).fit()           # running the linear model
print(lm.summary())


#### observations
1. R squared is 0.846
2. we do not see any significant change in R squared after dropping columns.
3. 'weekday-wed' has p-value 0.29
4. we now check for multicolinearity among the dependent variables

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# checking vif for multicolinearity
VIF1=pd.DataFrame()
VIF1['columns']=x_train3.columns
VIF1['vif']=list(map(lambda x:variance_inflation_factor(x_train3,x),range(x_train3.shape[1])))
VIF1.sort_values(by='vif',ascending=False)

#### OBSERVATIONS
'weathersit-moderate','temp','weathersit-good','hum','workingday' : these columns have VIF score greater 5\
**we will now drop these columns one by one and check the multicolinearity and performance of our model**


**dropping 'weathersit-moderate' column**

### Dropping the variable and updating the model

In [ ]:
x_train3.drop('weathersit-moderate',axis=1,inplace=True)     #dropping the columns weathersit-moderate

In [ ]:
x_train5=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train5).fit()       #running the linear model
print(lm.summary())

1. R squared is 0.833
2. we do not see any significant change in R squared after dropping columns.
3. we now check for multicolinearity among the dependent variables

In [ ]:
# checking vif for multicolinearity
VIF2=pd.DataFrame()
VIF2['columns']=x_train3.columns
VIF2['vif']=list(map(lambda x:variance_inflation_factor(x_train3,x),range(x_train3.shape[1])))
VIF2.sort_values(by='vif',ascending=False)

#### OBSERVATIONS
'temp','hum','workingday' : these columns have VIF score greater 5\
**'weekday-wed' has p-value 0.45**

**dropping 'weekday-wed	' column**

### Dropping the variable and updating the model

In [ ]:
x_train3.drop('weekday-wed',axis=1,inplace=True)    #dropping 'weekday-wed' column

In [ ]:
x_train6=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train6).fit()     # running the linear model
print(lm.summary())

#### OBSERVATIONS
1. R squared is 0.833
2. we do not see any significant change in R squared after dropping columns.
3. we now check for multicolinearity among the dependent variables

In [ ]:
# checking vif for multicolinearity
VIF3=pd.DataFrame()
VIF3['columns']=x_train3.columns
VIF3['vif']=list(map(lambda x:variance_inflation_factor(x_train3,x),range(x_train3.shape[1])))
VIF3.sort_values(by='vif',ascending=False)

#### OBSERVATIONS
'temp','hum','workingday' : these columns have VIF score greater 5\
**dropping 'workingday	' column and then creating a model**

### Dropping the variable and updating the model

In [ ]:
x_train3.drop('workingday',axis=1,inplace=True)    #dropping 'workingday' column

In [ ]:
x_train8=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train8).fit()       #running the linear model
print(lm.summary())

1. R squared is 0.832
2. we do not see any significant change in R squared after dropping columns.
3. we see high p-value for weekday-sat column (0.68)
4. we now check for multicolinearity among the dependent variables

In [ ]:
# checking vif for multicolinearity
VIF4=pd.DataFrame()
VIF4['columns']=x_train3.columns
VIF4['vif']=list(map(lambda x:variance_inflation_factor(x_train3,x),range(x_train3.shape[1])))
VIF4.sort_values(by='vif',ascending=False)

dropping 'weekday-sat' column as it has high p value

### Dropping the variable and updating the model

In [ ]:
# dropping 'weekday-sat' column as it has high p value
x_train3.drop('weekday-sat',axis=1,inplace=True)

In [ ]:
# creating another model using after dropping the column 'weekday-sat' 
x_train8=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train8).fit()       #running the linear model
print(lm.summary())

1. R squared is 0.832
2. we do not see any significant change in R squared after dropping column 'weekday-sat'.
3. we see p-value of all the dependent variables are less 0.05
4. we now check for multicolinearity among the dependent variables

In [ ]:
# checking vif for multicolinearity
VIF4=pd.DataFrame()
VIF4['columns']=x_train3.columns
VIF4['vif']=list(map(lambda x:variance_inflation_factor(x_train3,x),range(x_train3.shape[1])))
VIF4.sort_values(by='vif',ascending=False)

'temp','hum' : these columns have VIF score greater 5\
**dropping 'hum'  column and then creating a model**

### Dropping the variable and updating the model

In [ ]:
# dropping 'hum' column as it has high vif value
x_train3.drop('hum',axis=1,inplace=True)

In [ ]:
# creating another model using after dropping the column 'weekday-sat' 
x_train8=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train8).fit()       #running the linear model
print(lm.summary())

1. R squared is 0.817
2. we see high p-value for 'mnth-may' column (0.34)


### Dropping 'mnth-may' column and creating model again

In [ ]:
# dropping 'hum' column as it has high vif value
x_train3.drop('mnth-may',axis=1,inplace=True)

In [ ]:
# creating another model using after dropping the column 'weekday-sat' 
x_train8=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train8).fit()       #running the linear model
print(lm.summary())

1. R squared is 0.817
2. we see  p-value for 'mnth-sept' column greater 0.05

### Dropping 'mnth-sept' column and creating model again

In [ ]:
# dropping 'hum' column as it has high vif value
x_train3.drop('mnth-sept',axis=1,inplace=True)

In [ ]:
# creating another model using after dropping the column 'weekday-sat' 
x_train8=sm.add_constant(x_train3)
lm=sm.OLS(y_train,x_train8).fit()       #running the linear model
print(lm.summary())

In [ ]:
#we now check for multicolinearity among the dependent variables

In [ ]:
# checking vif for multicolinearity
VIF4=pd.DataFrame()
VIF4['columns']=x_train3.columns
VIF4['vif']=list(map(lambda x:variance_inflation_factor(x_train3,x),range(x_train3.shape[1])))
VIF4.sort_values(by='vif',ascending=False)

**we see all the dependent variables have vif in acceptable range**

In [ ]:
x_train3.columns

***Doing feature selection using RFE to compare the columns we got through manual process with the automated one***

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
rfe=RFE(estimator=lr,n_features_to_select=15)
rfe.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,rfe.ranking_,rfe.support_))

In [ ]:
cols=x_train.columns[rfe.support_]

In [ ]:
cols    # independent variables we got through rfe

In [ ]:
x_train3.columns

**So we see the set of independent variables we got though manual feature elimination is almost same as  that of 
RFE method**

In [ ]:
# features we chooose for prediction
x_train3.columns

# Residual Analysis of the train data

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
y_train_cnt = lm.predict(x_train8)

In [ ]:
sns.distplot(y_train['cnt']-y_train_cnt)
plt.xlabel('error')
plt.title('Distribution of error')
plt.show()

In [ ]:
# plotting qq-plot to check if the eroor is normally distributed
sm.qqplot(y_train['cnt']-y_train_cnt,fit=True,line='45')

### Check for Homoscedasticity

Homoscedasticity means that the residuals have equal or almost equal variance across the regression line. By plotting the error terms with predicted terms we can check that there should not be any pattern in the error terms

In [ ]:
sns.scatterplot(y_train_cnt,y_train_cnt-y_train['cnt'])
plt.xlabel('y_pred/predicted values')
plt.ylabel('Residuals')
plt.show()

# To evaluaate the model, we are going to check the r2 score 

In [ ]:
var=x_train3.columns    # columns we want to make predictions on.
n=x_test.shape[0]       
d=len(var)
test_x=x_test[var]  # filtering columns from test set on which we want to make predictions  

In [ ]:
test1=sm.add_constant(test_x)   #adding constant column to the test data set

In [ ]:
y_pred=lm.predict(test1)    # making prediction on the test set

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(y_test, y_pred)   #checking R2 score for test data
adj_r2=1-((1-r2)*(n-1))/(n-d-1)

In [ ]:
print(r2)
print(adj_r2)


# Model Evaluation

Let's now plot the graph for actual versus predicted values.

In [ ]:
fig = plt.figure()
sns.scatterplot(y_test['cnt'],y_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20)
plt.xlabel('y_test', fontsize = 18)                         
plt.ylabel('y_pred', fontsize = 16) 
plt.show()

# The equation of our best fitted line:

In [ ]:
lm.params

Target variable = 1894.15 + (season-spring * (-725.82)) + (season-winter * 533.92) + (mnth-dec * (-531.77)) + (mnth-feb * (-463.36)) + (mnth-jan * (-555.55)) + (mnth-jul * (-580.38)) + (mnth-nov * -572.84) + (weekday-mon * (-320.90))+ (weekday-tue *(-245.31)) + (weathersit-good * 727.62) + (yr * (2111.66)) + (holiday * (-799.94)) + (temp * (3616.33)) + (windspeed*(-1428.60))


From above coefficients of features having highest magnitude are the top features affecting our bikes demand.\
Top 5 of them are as follows
1. temp(coef =3616.32)
2. yr(Coefficient= 2111.65)
3. windspeed(coef=-1428.59)
4. holiday(coef= -799.93)
5. season-spring(coef= -725.82)

In [ ]:
# R squared score and Adjusted R squared score for x_test
print(r2)
print(adj_r2)